# Brains4Buildings interactive inverse grey-box analysis pipeline

This Jupyter Labs notebook can be used to interactively test the Brains4Buildings inverse grey-box analysis pipeline.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!

This file be downloaded from the [twomes-dataset-windesheim-brains4buildings2022 repository](https://edu.nl/cvwtj):
- `raw_properties/b4b_raw_properties.parquet`; 
- `metadata/b4b-room-metadata.zip`

## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

from tqdm.notebook import tqdm
from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

base_url = 'https://github.com/energietransitie/twomes-dataset-windesheim-brains4buildings2022/raw/main/'

#location: Google Maps location of Hogeschool Windesheim in Zwolle
lat, lon = 52.499255, 6.0765167
timezone_ids = 'Europe/Amsterdam'

from preprocessor import Preprocessor
from inversegreyboxmodel import Learner
from plotter import Plot
from styler import formatted_error_dataframe

%load_ext autoreload
%matplotlib inline
%matplotlib widget

import logging
logging.basicConfig(level=logging.ERROR, 
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename='log_b4b.txt',
                   )

### Load Measured Data from parquet file

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first, but then you have to run this code on the energietransitiewindesheim.nl server

file_prop = 'raw_properties/b4b_raw_properties.parquet'
df_prop = pd.read_parquet(base_url + file_prop, engine='pyarrow')

# df_prop = pd.read_parquet('https://github.com/energietransitie/twomes-dataset-windesheim-brains4buildings2022/raw/main/raw_properties/b4b_raw_properties.parquet',
#                           engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFramews are properly sorted
if not df_prop.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prop = df_prop.sort_index()  

In [ ]:
df_prop.index.unique(level='id').values

In [ ]:
df_prop.index.unique(level='source').values

In [ ]:
df_prop

In [ ]:
df_prop.info()

In [ ]:
df_prop.describe()

In [ ]:
# Count values and make pivot-style overview

# Group by 'source' and count non-null values
df_counts = df_prop.groupby('source').count().sort_index(axis=1, ascending=False)

# Calculate column totals and add as a new row
column_totals = df_counts.sum().rename('Total')
df_counts_with_totals = pd.concat([df_counts, column_totals.to_frame().T])

# Calculate row totals and add as a new column
row_totals = df_counts_with_totals.sum(axis=1).rename('Total')
df_counts_with_totals['Total'] = row_totals

# Replace 0 values with spaces or other indication
df_counts_with_totals.replace(0, '-', inplace=True)

df_counts_with_totals = df_counts_with_totals.T

# Define desired row and column order
desired_rows = ['co2__ppm', 'occupancy__p', 'valve_frac__0', 'temp_in__degC', 'rel_humidity__0',
                'occupancy__bool', 'window_open__bool', 'door_open__bool', 'Total']
desired_columns = ['CO2-meter-SCD4x', 'bms', 'xovis', 'human_observer', 'Total']

# Reindex and reorder rows and columns
df_counts_with_totals = df_counts_with_totals.reindex(desired_rows)
df_counts_with_totals = df_counts_with_totals[desired_columns]

# Transpose the dataframe
df_counts_with_totals = df_counts_with_totals.transpose()

# Display the dataframe
df_counts_with_totals


In [ ]:
# overview of CO₂ values
df_prop.filter(regex='__ppm$').describe().loc[['min', 'mean', 'max']].dropna(axis=1, how='all')

In [ ]:
# histogram of CO₂ measurements before preprocessing; also filter out simulated CO₂ concentration values (in case this cell is run after learning)
df_hist = df_prop.filter(regex='^(?!.*sim_co2__ppm$).*__ppm$')

labels = [col.split('_co2__')[0] for col in df_hist.columns]
df_hist.columns = labels
df_hist.plot.hist(bins=200, alpha=0.5)
plt.xlabel( 'CO₂ concentration [ppm]')

In [ ]:
# overview of CO₂ values, per room
df_prop.filter(regex='__ppm$').groupby(['id','source']).describe().loc[:, pd.IndexSlice[:, ['min', 'mean', 'max']]].stack(level=0).reindex(columns=['min', 'mean', 'max'])

In [ ]:
# histogram of CO₂ measurements per room before preprocessing; also filter out simulated CO₂ concentration values (in case this cell is run after learning)
df_hist = Preprocessor.unstack_prop(df_prop).filter(regex='^(?!.*sim_co2__ppm$).*__ppm$').dropna(axis=1, how='all').unstack([0])

df_hist.columns = df_hist.columns.swaplevel(0,1)

df_hist.columns = ['_'.join(map(str, col)) for col in df_hist.columns.values]

df_hist.columns.values

labels = [col.split('_co2__')[0] for col in df_hist.columns]
df_hist.columns = labels
df_hist.plot.hist(bins=200, alpha=0.5)
plt.xlabel( 'CO₂ concentration [ppm]')

## Preprocessing including merging weather data

In [ ]:
df_prep = Preprocessor.preprocess_room_data(df_prop, lat, lon, timezone_ids)

In [ ]:
df_prep.index.unique(level='id').values

In [ ]:
df_prep.info()

In [ ]:
df_prep.describe()

In [ ]:
# overview of CO₂ values
df_prep.filter(regex='__ppm$').describe().loc[['min', 'mean', 'max']].dropna(axis=1, how='all')

In [ ]:
# overview of CO₂ values, per room
df_prep.filter(regex='__ppm$').groupby('id').describe().loc[:, pd.IndexSlice[:, ['min', 'mean', 'max']]].stack(level=0).reindex(columns=['min', 'mean', 'max'])

In [ ]:
df_prep

In [ ]:
# visualize input data
df_plot = df_prep

In [ ]:
df_plot.info()

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$',
    'm_s_1': r'$m/s$',
    'W_m_2' : r'$W/m^2$'
}

In [ ]:
#Plot all properties from all sources for all ids but only for certain columns
Plot.dataframe_preprocessed_plot(df_plot.filter(regex='.*(__ppm$|valve_frac__0$|__p$|__m_s_1$)'), units_to_mathtext)

In [ ]:
# Plot a matrix of scatterplots for all measurements
features = ['CO2-meter-SCD4x_co2__ppm', 'bms_co2__ppm', 'bms_occupancy__bool', 'CO2-meter-SCD4x_occupancy__p', 'xovis_occupancy__p', 'bms_valve_frac__0']
Plot.features_scatter_plot(df=df_prep, features=features)

In [ ]:
df_prep.columns

In [ ]:
# histogram of CO₂ measurements after preprocessing; also filter  the columns to exclude simulated CO₂ concentrations (if this cell is run after learning)
df_hist = df_prep.filter(regex='^(?!.*sim_co2__ppm$).*__ppm$').dropna(axis=1, how='all')
labels = [col.split('_co2__')[0] for col in df_hist.columns]
df_hist.columns = labels
df_hist.plot.hist(bins=200, alpha=0.5)
plt.xlabel( 'CO₂ concentration [ppm]')

In [ ]:
# histogram of CO₂ measurements per room after preprocessing 
df_hist = df_prep.filter(regex='^(?!.*sim_co2__ppm$).*__ppm$').dropna(axis=1, how='all').unstack([0])

df_hist.columns = df_hist.columns.swaplevel(0,1)

df_hist.columns = ['_'.join(map(str, col)) for col in df_hist.columns.values]

df_hist.columns.values

labels = [col.split('_co2__')[0] for col in df_hist.columns]
df_hist.columns = labels

df_hist.plot.hist(bins=200, alpha=0.5)

plt.xlabel( 'CO₂ concentration [ppm]')

## Learn parameters using inverse grey-box analysis

Most of the heavy lifting is done by the `learn_room_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
%%time 
%autoreload 2

# learn the model parameters for only a subset of the room ids and write results to a dataframe

# read room metadata

file_metadata = 'metadata/b4b-room-metadata.zip'
df_room_metadata = pd.read_csv(base_url + file_metadata, usecols=['id', 'room__m3', 'vent_max__m3_h_1']).set_index(['id'])

# filename = '../data/b4b-room-metadata.zip'
# df_room_metadata = pd.read_csv(filename, usecols=['id', 'room__m3', 'vent_max__m3_h_1']).set_index(['id'])

hints = {
    'A_inf__m2' : (10 / 1e4) # 10 [cm^2] 
}
    
#select whether to learn a time-varying parameters as well (NB you can set 0 or 1 of these to true, both not both
learn = [
    'A_inf__m2',
    'valve_frac__0',
    # 'occupancy__p',
]

#select column names
property_sources = {
    'co2__ppm' : 'CO2-meter-SCD4x_co2__ppm',           # options: ['bms_co2__ppm', 'CO2-meter-SCD4x_co2__ppm']
    'occupancy__p': 'CO2-meter-SCD4x_occupancy__p',    # options: ['CO2-meter-SCD4x_occupancy__p', 'xovis_occupancy__p']
    'valve_frac__0': 'bms_valve_frac__0'
}

df_results_per_period, df_results = Learner.learn_room_parameters(df_prep,
                                                                  property_sources = property_sources, 
                                                                  df_metadata = df_room_metadata,
                                                                  hints = hints,
                                                                  learn = learn,
                                                                  learn_period__d = 3, 
                                                                  learn_change_interval__min = 30,
                                                                  co2_ext__ppm = 415,
                                                                  ev_type=2
                                                                 )


### Result per learning period 

In [ ]:
#write df_results_per_period to zipped CSV file for analysis in other programs
if any(df_results.columns.str.startswith('model_')): 
    roomtype='virtual'
else:
    roomtype='real'
    
if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
    if 'valve_frac__0' in learn:
        resulttype = 'valve_frac__0'
    else:
        resulttype = 'occupancy__p'
else:
    resulttype= 'A_inf__m2'

df_results_per_period.to_csv(f'results_{resulttype}_per_period_{roomtype}_rooms.zip',
                             encoding='utf-8',
                             compression= dict(method='zip',
                                               archive_name=f'results_{resulttype}_per_period_{roomtype}_rooms.csv'),
                             date_format='%Y-%m-%dT%H:%M:%S%z')

In [ ]:
df_results_per_period

In [ ]:
# box plot for 'learned_A_inf__cm2' per room 
Plot.learned_parameters_boxplot_b4b(df_results_per_period, learned='learned_A_inf__cm2', actual=None)


In [ ]:
# show essential statistics for the learned values
df_stats = df_results_per_period.describe().filter(regex='^actual_|^learned_')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
df_results_per_period

In [ ]:
# show essential statistics for the errors
formatted_error_dataframe(df_results_per_period, per_id= False)

In [ ]:
# show essential statistics for the error values, per id
formatted_error_dataframe(df_results_per_period, per_id= True)

### Result Visualization

In [ ]:
#Plot all properties from all sources for all ids
input_props = list(property_sources.values())
learned_props = input_props + ['sim_co2__ppm']
learned_props_frac = input_props + ['sim_co2__ppm', 'learned_valve_frac__0']
learned_props_occupancy = input_props + ['sim_co2__ppm', 'learned_occupancy__p']

In [ ]:
df_results

In [ ]:
# select properties to visualise
if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
    if 'valve_frac__0' in learn:
        props = learned_props_frac
    else:
        props = learned_props_occupancy
else:
    props = learned_props
        
# df_plot = df_prep[props]

df_plot = df_results[props]

In [ ]:
props

In [ ]:
df_plot

In [ ]:
df_plot.info()

In [ ]:
#Plot time series of all relevant measurements and learned properties
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
# Plot a matrix of scatterplots for all relevant measurements and learned properties
Plot.features_scatter_plot(df=df_prep, features=props)

## Learn parameters using inverse grey-box analysis with co2_ext__ppm 50 ppm lower 

Most of the heavy lifting is done by the `learn_room_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
# use the alt variable to turn on or off processing an alternative scenario and comparing it with the base case 
alt = False

In [ ]:
%%time 
%autoreload 2

if alt:
    df_results_per_period_margin_50, df_results_margin_50 = Learner.learn_room_parameters(df_prep,
                                                                      property_sources = property_sources, 
                                                                      df_metadata = df_room_metadata,
                                                                      hints = hints,
                                                                      learn = learn,
                                                                      learn_period__d = 3, 
                                                                      learn_change_interval__min = 30,
                                                                      co2_ext__ppm = 415-50,
                                                                      ev_type=2
                                                                     )


### Result per learning period with co2_ext__ppm 50 ppm lower

In [ ]:
#write df_results_per_period to zipped CSV file for analysis in other programs
if alt:
    if any(df_results.columns.str.startswith('model_')): 
        roomtype='virtual'
    else:
        roomtype='real'

    if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
        if 'valve_frac__0' in learn:
            resulttype = 'valve_frac__0'
        else:
            resulttype = 'occupancy__p'
    else:
        resulttype= 'A_inf__m2'

    df_results_per_period_margin_50.to_csv(f'results_{resulttype}_per_period_{roomtype}_rooms_margin_50__ppm.zip',
                                 encoding='utf-8',
                                 compression= dict(method='zip',
                                                   archive_name=f'results_{resulttype}_per_period_{roomtype}_rooms.csv'),
                                 date_format='%Y-%m-%dT%H:%M:%S%z')

In [ ]:
if alt:
    df_results_per_period_margin_50

In [ ]:
# box plot for 'learned_A_inf__cm2' per room 
if alt:
    Plot.learned_parameters_boxplot_b4b(df_results_per_period, learned='learned_A_inf__cm2', actual=None)

In [ ]:
# show essential statistics for the learned values
if alt:
    df_stats_margin_50 = df_results_per_period_margin_50.describe().filter(regex='^actual_|^learned_')
    df_stats_margin_50.loc[df_stats_margin_50.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the errors
if alt:
    formatted_error_dataframe(df_results_per_period_margin_50, per_id= False)

In [ ]:
# show essential statistics for the error values, per id
if alt:
    formatted_error_dataframe(df_results_per_period_margin_50, per_id= True)

### Result Visualization with co2_ext__ppm 50 ppm lower

In [ ]:
#Plot all properties from all sources for all ids
if alt:
    input_props = list(property_sources.values())
    learned_props = input_props + ['sim_co2__ppm']
    learned_props_frac = input_props + ['sim_co2__ppm', 'learned_valve_frac__0']
    learned_props_occupancy = input_props + ['sim_co2__ppm', 'learned_occupancy__p']

In [ ]:
if alt:
    df_results_margin_50

In [ ]:
# select properties to visualise
if alt:
    if (('valve_frac__0' in learn) or ('occupancy__p'in learn)):
        if 'valve_frac__0' in learn:
            props = learned_props_frac
        else:
            props = learned_props_occupancy
    else:
        props = learned_props

    # df_plot = df_prep[props]

    df_plot = df_results_margin_50[props]

In [ ]:
if alt:
    props

In [ ]:
if alt:
    df_plot

In [ ]:
if alt:
    df_plot.info()

In [ ]:
#Plot time series of all relevant measurements and learned properties
if alt:
    Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
# Plot a matrix of scatterplots for all relevant measurements and learned properties
if alt:
    Plot.features_scatter_plot(df=df_prep, features=props)

## Improvements with co2_ext_ppm 50 ppm lower compared to 1 ppm lower

In [ ]:
if alt:
    df_compare = pd.DataFrame(columns=['learn_streak_period_start', 'learn_streak_period_end', 'mae_co2__ppm', 'rmse_co2__ppm', 'mae_valve_frac__0', 'rmse_valve_frac__0', 'mae_occupancy__p', 'rmse_occupancy__p'])

In [ ]:
if alt:
    df_compare[['learn_streak_period_start', 'learn_streak_period_end']] = df_results_per_period[['learn_streak_period_start', 'learn_streak_period_end']]

In [ ]:
if alt:
    metrics = ['mae_co2__ppm', 'rmse_co2__ppm', 'rmae_valve_frac__0', 'rmse_valve_frac__0', 'mae_occupancy__p', 'rmse_occupancy__p']
    df_compare[metrics]= df_results_per_period[metrics] - df_results_per_period_margin_50[metrics]


In [ ]:
if alt:
    df_compare

In [ ]:
# show essential statistics for the errors
if alt:
    df_improvements_when_co2_margin_50__ppm = df_compare.describe().filter(regex='^mae_|^rmae_|^rmse')
    df_improvements_when_co2_margin_50__ppm.loc[df_improvements_when_co2_margin_50__ppm.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]
